In [65]:
#installation of VectorBT
pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
#installation of Yfinance 
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
# importing necessary libraries
import vectorbt  as vbt
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
#import numba

In [68]:
# Fetch historical data

# determine variable 'symbol' for fetching many tickets simultaneously (at same time)
symbol = [
    'BTC-USD', 'ETH-USD', 'XRP-USD', 'BCH-USD', 'LTC-USD'#,
]
"""    'BNB-USD', 'EOS-USD', 'XLM-USD', 'XMR-USD', 'ADA-USD',
    'QBIT-USD', 'TSLA', 'AMZN', 'AAPL', 'DGTX-USD', 'BTG-USD', 
    'LINK-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD',
    'TON-USD', 'OKB-USD', 'GLD', 'GIS', 'PG', 'MRK', 'AMT', 'BMY',
    'CVX', 'S', 'CVS', 'HD', 'PUK'
#]
"""

# "Attach" dataset prices (dataframe) to variable ohlcv and at the same time download dataset from Yahoo Finance 
ohlcv = yf.download(symbol, interval="5m", period="60d")

# "Attach" only closing prices from dataset to variable 'price'
price = ohlcv['Close']

# "Attach" only High prices from dataset to variable 'high_price'
high_price = ohlcv['High']

# Same for low
low_price = ohlcv['Low']

[*********************100%***********************]  5 of 5 completed


In [69]:
# Define the RSI indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics 
def rsi(price_set, rsi_window=14):
        delta = price_set.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(rsi_window).mean()
        avg_loss = loss.rolling(rsi_window).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))


# Define Divergence 
def divergence(
# using price set as starting point to calculate divergence 
  price_set,
# using last 14 bars/days period window for RSI indicator. This is just a template and can be changed when we will execute .run function in next stage.
  rsi_window=14,
# using last 1 bar/day period window for fast Moving Average indicator. This is just a template and can be changed when we will execute .run function in next stage.
  fast_ma_window = 1,

# using last 50 bars/days period window for fast Moving Average indicator.  This is just a template and can be changed when we will execute .run function in next stage.
  slow_ma_window = 50):
   

# Calculate the RSI indicator for using in calculation of divergence 
   rsi_indicator = rsi(price_set)
#   rsi = vbt.RSI.run(price, window = rsi_window).rsi.to_numpy()



# Calculate the Divergence indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics 
   bullish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() > 0))
   
   bearish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() < 0))
   #print((ohlcv['Close']).diff()<0)
   
   # Get fast Moving Average (MA) of the prices
   fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
   print(type(bearish_divergence))
   # Get slow Moving Average (MA) of the prices
   slow_ma = vbt.MA.run(price_set, slow_ma_window, short_name='slow')
   print(type(slow_ma))

   # Determine UpTrend (fast MA above slow MA)
   trend_up = fast_ma.ma_above(slow_ma)   

   # Determine DownTrend (fast MA below slow MA)
   trend_down = fast_ma.ma_below(slow_ma)

   # Convert everything to Numpy array 
   trend_up = trend_up.to_numpy()
   trend_down = trend_down.to_numpy()
   bullish_divergence = bullish_divergence.to_numpy()
   bearish_divergence = bearish_divergence.to_numpy()
   print(type(bearish_divergence))

   # Combine all previous indicators into the one 
   # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
   indicator = np.where( ( ( (bullish_divergence) == True)  & (trend_up == True) ), 1, 0)
   indicator = np.where( ( ( (bearish_divergence) == True)  & (trend_down == True) ), -1, indicator)
   
   return indicator


# Giving recipe to Indicator Factory for our custom indicator
indicator_divergence_ma = vbt.IndicatorFactory(
   class_name = 'Combination_Divergence_MA',
   short_name = 'DIVER_MA',
   input_names = ['price'],
   param_names = ['rsi_window', 'fast_ma_window', 'slow_ma_window'], 
   output_names = ['value'],
   ).from_apply_func(
       divergence,
        rsi_window = dict(is_array_like=True, bc_to_input=1, per_column=True),
        fast_ma_window = dict(is_array_like=True, bc_to_input=1, per_column=True),
        slow_ma_window = dict(is_array_like=True, bc_to_input=1, per_column=True),       
    #   rsi_window = 14, 
    #   fast_ma_window = 1,
    #   slow_ma_window = 50,
        keep_pd = True
       )


   # Get ATR
atr = vbt.ATR.run(high_price, low_price, price)
#atr = atr.atr.to_numpy()

atr = atr.atr.to_numpy()
##print(type(atr))

# Calculate adaptive/dynamically changing value based on ATR and current price 
adaptive_window = atr * 1000 / price #* rsi_window
adaptive_window = adaptive_window * 10
#adaptive_window = adaptive_window.values.tolist()
#adaptive_window = adaptive_window.to_numpy()

###print(adaptive_window)

#@jit(nopython=True)
#@jit(nopython=True, parallel=True) 

results = indicator_divergence_ma.run(
   price,
   rsi_window = 14,   
   fast_ma_window = 1,
   slow_ma_window = 50,
  # per_column = True
   )

##print ((results.value))
# Determine point of entries based on signals 
entries = results.value == 1.0
exits = results.value == -1.0
##print(type(entries))




# "Run" our signals through the history (dataset) and count the results
pf = vbt.Portfolio.from_signals(price, entries, exits)

# Выводим общие результаты 
print(pf.total_return())
print(pf.stats())

<class 'pandas.core.frame.DataFrame'>
<class 'vectorbt.indicators.basic.MA'>
<class 'numpy.ndarray'>
DIVER_MA_rsi_window  DIVER_MA_fast_ma_window  DIVER_MA_slow_ma_window         
14                   1                        50                       BCH-USD    0.354566
                                                                       BTC-USD    0.469765
                                                                       ETH-USD    0.457173
                                                                       LTC-USD    0.517085
                                                                       XRP-USD    0.007222
Name: total_return, dtype: float64
Start                         2023-01-20 00:00:00+00:00
End                           2023-03-20 23:25:00+00:00
Period                                 59 days 23:30:00
Start Value                                       100.0
End Value                                      136.1162
Total Return [%]                                36

<ipython-input-69-2f6b9fa60f3f>:124: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x7f87240fedc0>. Pass column to select a single column/group.
  print(pf.stats())
